In [ ]:
import sys
from pathlib import Path

import pandas as pd
import folium
from folium.plugins import TimestampedGeoJson
import seaborn as sns
from matplotlib import cm, colors

# make project root discoverable
project_root = Path(__file__).resolve().parents[2] if "__file__" in globals() else Path().resolve().parents[1]
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from paths import DATA_DIR


In [ ]:
data = pd.read_parquet(DATA_DIR / "train.parquet")
data_test = pd.read_parquet(DATA_DIR / "final_test.parquet")

data.head()


In [ ]:
m = folium.Map(location=data[["latitude", "longitude"]].mean(axis=0), zoom_start=13)

for _, row in (
    data[["counter_name", "latitude", "longitude"]]
    .drop_duplicates("counter_name")
    .iterrows()
):
    folium.Marker(
        row[["latitude", "longitude"]].values.tolist(), 
        popup=row["counter_name"]
    ).add_to(m)

m


In [ ]:
grouped_data = (
    data.groupby(["counter_name", pd.Grouper(freq="1M", key="date")])["log_bike_count"]
    .sum()
    .reset_index()
)

coordinates_mapper = data[["counter_name", "latitude", "longitude"]].drop_duplicates()
grouped_data = grouped_data.merge(
    coordinates_mapper,
    on="counter_name",
    how="left"
)

grouped_data.head()


In [ ]:
# Normalize log_bike_count for gradient mapping
log_min, log_max = grouped_data["log_bike_count"].min(), grouped_data["log_bike_count"].max()
norm = colors.Normalize(vmin=log_min, vmax=log_max)

# Create a colormap (green → yellow → red, reversed for high=red)
cmap = cm.get_cmap("RdYlGn_r")

def get_gradient_color(log_value):
    rgba_color = cmap(norm(log_value))
    return colors.rgb2hex(rgba_color[:3])


In [ ]:
features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["longitude"], row["latitude"]],
        },
        "properties": {
            "time": row["date"].isoformat(),
            "icon": "circle",
            "iconstyle": {
                "fillColor": get_gradient_color(row["log_bike_count"]),
                "fillOpacity": 1,
                "stroke": "false",
                "radius": 8,
            },
            "style": {"weight": 0},
            "popup": f"{row['counter_name']} — Bike Count: {row['log_bike_count']}",
        },
    }
    for _, row in grouped_data.iterrows()
]

geojson_data = {
    "type": "FeatureCollection",
    "features": features,
}


In [ ]:
m = folium.Map(location=data[["latitude", "longitude"]].mean(axis=0), zoom_start=13)

TimestampedGeoJson(
    data=geojson_data,
    period="P1M",              # Monthly steps
    add_last_point=False,
    auto_play=True,
    loop=True,
    max_speed=10,
    loop_button=True,
    date_options="YYYY-MM",
    time_slider_drag_update=True,
).add_to(m)

m
